In [98]:
import pandas as pd
import numpy as np
from urllib.request import urlopen



In [99]:
provincias = pd.read_csv('raw_data/provincias.csv')
energy = pd.read_csv('raw_data/Spain_energy_df_2017.csv')

In [100]:
provincias_grouped = provincias.groupby('Provincia').agg({'latitude':'mean','longitude':'mean'}).reset_index()

In [101]:
def get_closest_province(row):
    df = provincias_grouped[['latitude','longitude']] - list(row[['lat','lon']])

    df['agg'] = df['latitude']**2 + df['longitude']**2
    name_index = df.sort_values('agg').iloc[0,:].name

    return provincias_grouped.iloc[name_index].Provincia
energy['province'] = energy.apply(get_closest_province,axis=1)

energy.groupby('province').agg({'Solar Generation (2017)':'sum'}).sort_values('Solar Generation (2017)',ascending=False).head(3)

,Solar Generation (2017)
province,
Tarragona,29594.68
Caceres,18406.04
Valencia/Val�ncia,15133.59


In [102]:
energy.to_csv('Energy_Province.csv',index=False)

In [103]:
energy

,Unnamed: 0,country,capacity_mw,lat,lon,primary_fuel,Solar Generation (2017),province
0,20135,ESP,877.660,43.5528,-5.7231,Coal,4003.64,Asturias
1,20136,ESP,758.740,39.9427,-3.8548,Gas,1798.46,Toledo
2,20137,ESP,15.000,41.5485,0.8245,Waste,5.22,Lleida
3,20138,ESP,18.000,41.8559,-1.9224,Wind,50.55,Soria
4,20139,ESP,16.334,41.8559,-1.9224,Waste,5.68,Soria
...,...,...,...,...,...,...,...,...
824,20959,ESP,9.000,41.6428,-4.8151,Solar,15.62,Valladolid
825,20960,ESP,7.600,39.8710,-6.8320,Solar,14.92,Caceres
826,20961,ESP,9.000,41.9340,-0.8050,Solar,16.03,Zaragoza
827,20962,ESP,9.900,41.9440,-0.8180,Solar,17.37,Zaragoza
